## Capstone Project
[**Gen AI Intensive 2025Q1**](https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1)


*By Pushpam Kumari*

[](https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1)

## 📘 Project Title
 ## Smart Study Buddy – Gen AI Learning Assistant using Gemini API
 An AI-Powered Personalized Learning Assistant using Gemini API!


## 🧠 Problem Statement
Students often find it difficult to digest lengthy chapters, revise effectively before exams, or get quick help with understanding complex concepts. Traditional learning resources don't always offer personalized explanations or tailored practice questions. This leads to wasted time, low retention, and academic stress.



## 🎯 Use Case

# 📚✨ Smart Study Buddy – Gen AI Learning Assistant using Gemini API

This notebook showcases a Gen AI-powered personalized learning assistant designed to help students:

- 📖 Summarize study material and long chapters  
- 💡 Explain tough concepts in a simple way  
- ❓ Generate quiz questions for practice and revision  

Built using Google’s Gemini model via the Python SDK, this project demonstrates multiple Gen AI capabilities working together to solve real academic challenges.  
With features like summarization, question-answering, and quiz generation, it acts like a smart tutor tailored to the learner's needs.
This tool acts like a digital tutor, available 24/7, to make studying more efficient, personalized, and interactive — especially during last-minute revisions or when preparing for exams.


## ⚙️ Setup and Installation

The following cells install and configure the required libraries and APIs to use the Google Generative AI SDK.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip uninstall -qy jupyterlab # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0" "chromadb==0.6.3"

In [ ]:
from google import genai
from google.genai import types


from IPython.display import HTML, Markdown, display
genai.__version__

In [ ]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
from google.generativeai import configure, GenerativeModel

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")


In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)


## 🧠 GenAI Capabilities Used

### ✨ Capability 1:Text Generation (Summarization)
* What it does: Takes a long piece of text and generates a concise summary based on the content.

* How I used it: I gave the Gemini model a paragraph (news article), and it generated a summary using generate_content().

## 🛠️ Implementation

We will pass a custom prompt to the Gemini model asking it to summarize a paragraph. Here's an example.


In [ ]:
paragraph = """
Today, Kapil Sibal, Senior Advocate and President of the Supreme Court Bar Association, addressed a press conference on the Vice President of India, Jagdeep Dhankhars remarks that the Supreme Court was using Article 142 as a nuclear missile against democratic forces.

Sibal said he is deeply saddened that a constitutional functionary is making such remarks.

"When I woke up this morning and read the Vice Presidents remarks in newspapers, I felt deeply saddened and shocked. If there is one institution even today which the people trust, it is the judicial institutions, be it the Supreme Court or the High Courts. Its truly concerning how some government officials respond to judicial decisions, when the verdict suits them, for instance, Article 370 abrogation or the Ram Janbhoomi judgment, they cite it as the Supreme Courts wisdom. But the moment a judgment doesnot align with their views, for instance, the recent Justice Pardiwalas judgment, they start levelling accusations."
"""

summary_prompt = f"Summarize the following paragraph:\n\n{paragraph}"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=summary_prompt
)

print("🔍 Summary:\n", response.text)


## 📤 Output

The model generates a short summary of the given text using its generative capabilities.

## 📖 Explanation

The `generate_content()` method is used to pass a summarization prompt to the model. The model processes the input and returns a concise version of the paragraph, demonstrating the ability to understand and condense content contextually.


### ✨ Capability 2: Semantic Understanding + Q&A (RAG-style

In this section, we demonstrate how Gemini can understand a given article and respond to questions based on it. 
This capability is useful for extracting insights from news content or any long-form documents using the 
`generate_content()` method of the Gemini API. This mirrors the *Retrieval-Augmented Generation (RAG)* 
concept where questions are answered using reference material.


## 🛠️ Implementation
We will pass a custom prompt to the Gemini model asking it to answer the user's question. Here's an example.

In [ ]:
question = "What did Kapil Sibal say about the judiciary in response to the Vice President's remarks?"

prompt = f"""
You are a helpful assistant that uses the passage below to answer the user's question.
Only use the content from the passage to answer the question. If the information is not present, say so.

PASSAGE:
{paragraph}

QUESTION:
{question}
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

print("🧠 Answer:\n", response.text)


## 📤 Output

The model generates an answer of the given query using its generative capabilities.

## 📖 Explanation

The `generate_content()` method is used to pass a summarization prompt to the model. The model processes the input and returns a concise version of the paragraph, demonstrating the ability to understand and condense content contextually.


### ✨ Capability 3:Structured Output with JSON
In this section, we use Gemini to generate multiple-choice questions (MCQs) based on a provided paragraph. We also instruct the model to format the output in JSON so that it can be easily parsed or integrated into educational apps and platforms.

Structured outputs are useful when building systems that need to programmatically use the output — such as a quiz app or LMS (Learning Management System).



## 🛠️ Implementation
We will pass a custom prompt to the Gemini model asking it to answer the user's question. Here's an example.

In [ ]:


quiz_prompt = f"""
Generate 4 multiple-choice quiz questions based on the passage below.
Each question should have:
- A "question" field (string)
- An "options" field (list of 4 strings)
- A "correct_option" field (one of "A", "B", "C", or "D")

Respond in JSON format only.

passage:{paragraph}
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=quiz_prompt
)

##print(response.text)
Markdown(response.text)


## 📤 Output
The Gemini model generates a short quiz in a structured format based on the input paragraph. Each question is relevant to the content, allowing learners to quickly test their understanding.

## 📖 Explanation
This section demonstrates the **Quiz Generation** capability of Gemini. By using the `generate_content()` method with a customized prompt, the model extracts key ideas from the input text and transforms them into **multiple-choice questions**.

Such a feature is highly beneficial for:

- **Students** who want quick revision tools  
- **Teachers** looking to generate quizzes from their study material  
- **EdTech apps** that aim to personalize learning experiences

Additionally, we formatted the output in Markdown to ensure the questions are displayed cleanly and are easy to read, making the tool more user-friendly and practical for real-world educational use cases.


## ✅ Conclusion

In this project, we built a simple yet powerful tool for students using Google’s Gemini model.  
By leveraging multiple Gen AI capabilities — summarization, question-answering, and quiz generation — we created a personalized learning assistant that makes studying more efficient and engaging.

This project not only demonstrates the practical applications of Generative AI in education but also opens up possibilities for further innovation, such as integrating with web platforms or expanding to other types of learning content.

Generative AI has the potential to become a student's best companion — and this is just the beginning.


## 🔮 Future Scope
- Enhance the tool to accept inputs from various sources such as **PDFs, Word documents, or YouTube video transcripts** for broader usability.
- Leverage additional Gemini capabilities such as **embeddings** for document retrieval or **multi-turn chat** for more interactive Q&A sessions.
- Integrate the solution into a simple and intuitive **web application** using frameworks like **Flask** or **Streamlit** to make it accessible to users without technical knowledge.